# Instructions:
1) In 'Specify Input/ Output Directories' code block, update: INPUT_PATH_FOLDER and OUTPUT_PATH_FOLDER
2) Run all cells in notebook

If you are having issue (or are working on this code) see the developer notes section at the end

In [1]:
# TODO: Add developer notes

## Import Libraries/ Functions:

In [2]:
#general library:
import os
import shutil
import numpy as np

#add sextractor to path
import sys
sys.path.insert(0, '../gibbi')

#now we can import the star_removal method:
from galaxy import *
from file_helper import read_galaxy_csv
from fits import view_fits
from threshold import iteratively_threshold_band_with_percentile
from gibbi import run_gibbi_on_galaxy
from visualize_panstarr_gibbi_output import run_visualize
from spin_parity import read_spin_parity_galaxies_label_from_csv

## Specify Input/ Output Directories:

In [3]:
################################  IMPORTANT ################################:
#modify these paths:

#path to folder (as it appears on your native OS)

CSV_PATH = '../../spin-parity-catalog/output_from_running/SpArcFiRe/table2/galaxy.csv'

INPUT_PATH_FOLDER = '../../spin-parity-catalog/output_from_running/sextractor/table2'
OUTPUT_PATH_FOLDER = '../../spin-parity-catalog/image_processing/star_mask/table2'
FITS_PATH_FOLDER =  '../../spin-parity-catalog/galaxies/table2'

#########################################################################

In [4]:
#TODO: Add check for folders

In [5]:
################################  IMPORTANT ################################:
#modify these paths:

def color_image_path(name):
    return '../../spin-parity-catalog/galaxies/table2/{}/{}_color.jfif'.format(name,name)

def fits_path(name,band):
    return '../../spin-parity-catalog/image_processing/preprocess_fits/table2/{}/{}_{}_preprocessed.fits'.format(name,name,band)

def bad_pixel_mask_path(name,band):
    return '../../spin-parity-catalog/image_processing/bad_pixel_mask/table2/{}/{}_{}_bad_pixel_mask.fits'.format(name,name,band)

def star_mask_path(name,band):
    return '../../spin-parity-catalog/image_processing/star_mask/table2/{}/{}_{}_star_mask.fits'.format(name,name,band)

visualize_path = '../../../gibbi_output/run_07_05_2022/table2'
spin_parity_path = '../../spin-parity-catalog/table_info/csv_format_of_table/table_2.csv'

#########################################################################

## Make star mask for all fits (nested) in INPUT_PATH_FOLDER:

In [6]:
csv_as_dict = read_galaxy_csv(CSV_PATH)

In [7]:
dark_side_dict = read_spin_parity_galaxies_label_from_csv(spin_parity_path)
dark_side_dict['IC 2101'] = dark_side_dict['IC2101'] #to fix issue in table2

In [8]:
def run_on_galaxy(gal_name,gal_dict,dark_side):
    try:
        print("Runnin on {}".format(gal_name))
        gal_color_image_path = color_image_path(gal_name)
    
        the_gal = galaxy(gal_name,gal_dict,gal_color_image_path,fits_path,bad_pixel_mask_path,star_mask_path)
        the_gal.create_masks()
    
        if len(the_gal.valid_bands) != 0:
            (xs,diff_dict,el_data_threshold_dict,valid_pixel_mask_dict) = run_gibbi_on_galaxy(the_gal)
            run_visualize(xs,diff_dict,el_data_threshold_dict,valid_pixel_mask_dict,the_gal,visualize_path,dark_side)
    except Exception as e:
        print(e)
    finally:
        del the_gal

In [9]:
DONE = os.listdir(visualize_path)

for gal_name in csv_as_dict:
    if gal_name in DONE:
        continue
    #gal_name = "NGC5395"
    gal_dict = csv_as_dict[gal_name]
    
    dark_side = ""
    if gal_name in dark_side_dict:
        dark_side = dark_side_dict[gal_name]
        
    run_on_galaxy(gal_name,gal_dict,dark_side)

Runnin on IC1755
Runnin on IC 2101
Runnin on IC5376
Runnin on MCG-02-02-030
Runnin on MCG-02-51-004
